In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import geopandas as gpd
from geopandas.tools import sjoin

from geojson import Point, Feature, FeatureCollection, dump

import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.cluster import KMeans  

from sklearn.metrics import silhouette_score

from scipy.spatial.distance import cdist

from shapely.geometry import Point
from shapely.geometry import MultiPoint
from shapely.geometry.polygon import Polygon
from shapely.ops import nearest_points

%matplotlib inline

plt.rcParams['figure.figsize'] = [15, 8]
plt.rcParams['figure.dpi'] = 100
plt.style.use('ggplot')

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 100)

In [2]:
ph_cluster_gmm = gpd.read_file("ph_clustered_3.geojson")
ph_cluster_gmm.head()

,ADM3_EN,ADM3_PCODE,ADM3ALT1EN,ADM2_EN,ADM1_EN,ADM0_EN,mean,total_area,nolit_per,lit_per,GHI,TEMP,PVout,Population,Cluster,geometry
0,Panglima Sugala,PH157001000,Balimbing,Tawi-Tawi,Autonomous Region in Muslim Mindanao,Philippines (the),0.830743,1.902365,84.332040,15.667960,1768.755191,26.736462,1295.129702,20542.154730,1,"MULTIPOLYGON (((119.93158 4.90053, 119.93167 4..."
1,Bongao,PH157002000,Capital,Tawi-Tawi,Autonomous Region in Muslim Mindanao,Philippines (the),2.654615,3.169599,26.215390,73.784610,1772.737411,26.965278,1304.740250,52269.208894,1,"MULTIPOLYGON (((119.76332 5.01178, 119.76282 5..."
2,Mapun,PH157003000,Cagayan de Tawi-Tawi,Tawi-Tawi,Autonomous Region in Muslim Mindanao,Philippines (the),0.554663,1.319500,78.153373,21.846627,1798.931213,26.727381,1339.543991,16606.520103,1,"MULTIPOLYGON (((118.40042 6.87783, 118.40052 6..."
3,Simunul,PH157004000,None,Tawi-Tawi,Autonomous Region in Muslim Mindanao,Philippines (the),0.753632,1.107767,16.713623,83.286377,1859.880903,27.089063,1438.417638,25882.470081,1,"MULTIPOLYGON (((119.84475 4.81262, 119.84490 4..."
4,Sitangkai,PH157005000,None,Tawi-Tawi,Autonomous Region in Muslim Mindanao,Philippines (the),1.306350,0.841592,34.511103,65.488897,1830.444744,27.094286,1393.338495,6919.100047,1,"MULTIPOLYGON (((119.38935 4.61459, 119.38918 4..."


In [3]:
pv_out_per_cluster_summary = ph_cluster_gmm.groupby('Cluster').agg({'PVout': ['mean', 'min', 'max','count']})
pv_out_per_cluster_summary

PVout                                
                mean          min          max count
Cluster                                             
1        1284.759495     1.464666  1738.880386  1591
2        1216.371006  1197.742694  1225.686045     3
3        1334.738071  1085.202384  1662.750582    53

In [4]:
lit_per_cluster_summary = ph_cluster_gmm.groupby('Cluster').agg({'lit_per': ['mean', 'min', 'max','count']})
lit_per_cluster_summary

lit_per                        
              mean        min    max count
Cluster                                   
1        44.578513   0.007890  100.0  1591
2        95.929860  87.789581  100.0     3
3        75.221216   0.783909  100.0    53

In [5]:
pop_per_cluster_summary = ph_cluster_gmm.groupby('Cluster').agg({'Population': ['mean', 'min', 'max','count']})
pop_per_cluster_summary

Population                                  
                 mean           min           max count
Cluster                                                
1        4.399156e+04  1.003416e+00  2.323185e+05  1591
2        2.068709e+06  1.581144e+06  3.035152e+06     3
3        4.325054e+05  2.438636e+05  9.997140e+05    53

In [6]:
doe_approved_projects = pd.read_csv('data/Approved Renewable Project of DOE as of Dec 2020 - Updated.csv', index_col = 0)
doe_approved_projects.head()

,connection_type,facility_name,category,subcategory,type,installed_capacity_mw,dependable_capacity_mw,number_of_units,province,municipality,latitude,longitude,region,operator,owner_ippa,fit_approved_for_re,owner_type,type_of_contract,date_commisioned_commercial,commisioned/commercial_year,c_kwh,mw_to_kw,no_of_hours,installed_g_co2_eq,dependable_g_co2_eq,icon,now,accumulated_installed_g_co2_eq,accumulated_dependable_g_co2_eq,operating_hours,id,suggested_area
island_grid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Luzon,Off-Grid,AGUADA*,Oil-Based,Diesel,NaN,0.012,0.011,NaN,Masbate,"Cataingan, Masbate",11.986010,123.799202,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,Aug-2010,2010.0,850.0,1000.0,8760.0,89352000,81906000,NaN,2021,982872000,900966000,5,16f63e94-236e-43ce-8388-4f0e7ba37f17,True
Luzon,Off-Grid,AMUTAG MGE1*,Oil-Based,Diesel,NaN,0.012,0.011,NaN,Masbate,"Aroroy, Masbate",12.468981,123.187455,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,Dec-2009,2009.0,850.0,1000.0,8760.0,89352000,81906000,NaN,2021,1072224000,982872000,5,2aadbc74-679d-4101-9058-38c8627b2185,True
Luzon,Off-Grid,AMUTAG MGE2*,Oil-Based,Diesel,NaN,0.012,0.011,NaN,Masbate,"Aroroy, Masbate",12.468981,123.187455,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,Dec-2009,2009.0,850.0,1000.0,8760.0,89352000,81906000,NaN,2021,1072224000,982872000,5,f4d4b05e-ac03-43cf-9130-281b5bf76833,True
Luzon,Off-Grid,BAANG(MOBO)*,Oil-Based,Diesel,NaN,0.006,0.005,NaN,Masbate,"Mobo, Masbate",12.264440,123.549202,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,May-2010,2010.0,850.0,1000.0,8760.0,44676000,37230000,NaN,2021,491436000,409530000,5,2a51f5c9-f992-4556-8ecf-ea39535907cc,True
Luzon,Off-Grid,BAAO*,Oil-Based,Diesel,NaN,0.006,0.005,NaN,Masbate,"Baleno, Masbate",12.409210,123.381844,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,Aug-2009,2009.0,850.0,1000.0,8760.0,44676000,37230000,NaN,2021,536112000,446760000,5,e06a22cd-a7f3-486a-866b-889e49bc60a2,True


In [7]:
gdf = gpd.GeoDataFrame(
    doe_approved_projects, geometry=gpd.points_from_xy(doe_approved_projects.longitude, 
                                                             doe_approved_projects.latitude))

gdf.head()

,connection_type,facility_name,category,subcategory,type,installed_capacity_mw,dependable_capacity_mw,number_of_units,province,municipality,latitude,longitude,region,operator,owner_ippa,fit_approved_for_re,owner_type,type_of_contract,date_commisioned_commercial,commisioned/commercial_year,c_kwh,mw_to_kw,no_of_hours,installed_g_co2_eq,dependable_g_co2_eq,icon,now,accumulated_installed_g_co2_eq,accumulated_dependable_g_co2_eq,operating_hours,id,suggested_area,geometry
island_grid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Luzon,Off-Grid,AGUADA*,Oil-Based,Diesel,NaN,0.012,0.011,NaN,Masbate,"Cataingan, Masbate",11.986010,123.799202,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,Aug-2010,2010.0,850.0,1000.0,8760.0,89352000,81906000,NaN,2021,982872000,900966000,5,16f63e94-236e-43ce-8388-4f0e7ba37f17,True,POINT (123.79920 11.98601)
Luzon,Off-Grid,AMUTAG MGE1*,Oil-Based,Diesel,NaN,0.012,0.011,NaN,Masbate,"Aroroy, Masbate",12.468981,123.187455,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,Dec-2009,2009.0,850.0,1000.0,8760.0,89352000,81906000,NaN,2021,1072224000,982872000,5,2aadbc74-679d-4101-9058-38c8627b2185,True,POINT (123.18745 12.46898)
Luzon,Off-Grid,AMUTAG MGE2*,Oil-Based,Diesel,NaN,0.012,0.011,NaN,Masbate,"Aroroy, Masbate",12.468981,123.187455,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,Dec-2009,2009.0,850.0,1000.0,8760.0,89352000,81906000,NaN,2021,1072224000,982872000,5,f4d4b05e-ac03-43cf-9130-281b5bf76833,True,POINT (123.18745 12.46898)
Luzon,Off-Grid,BAANG(MOBO)*,Oil-Based,Diesel,NaN,0.006,0.005,NaN,Masbate,"Mobo, Masbate",12.264440,123.549202,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,May-2010,2010.0,850.0,1000.0,8760.0,44676000,37230000,NaN,2021,491436000,409530000,5,2a51f5c9-f992-4556-8ecf-ea39535907cc,True,POINT (123.54920 12.26444)
Luzon,Off-Grid,BAAO*,Oil-Based,Diesel,NaN,0.006,0.005,NaN,Masbate,"Baleno, Masbate",12.409210,123.381844,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,Aug-2009,2009.0,850.0,1000.0,8760.0,44676000,37230000,NaN,2021,536112000,446760000,5,e06a22cd-a7f3-486a-866b-889e49bc60a2,True,POINT (123.38184 12.40921)


In [8]:
gdf.shape

(624, 33)

In [9]:
# Reproject point coordinates from DOE 
gdf.set_crs(epsg=4326, inplace=True).head()

,connection_type,facility_name,category,subcategory,type,installed_capacity_mw,dependable_capacity_mw,number_of_units,province,municipality,latitude,longitude,region,operator,owner_ippa,fit_approved_for_re,owner_type,type_of_contract,date_commisioned_commercial,commisioned/commercial_year,c_kwh,mw_to_kw,no_of_hours,installed_g_co2_eq,dependable_g_co2_eq,icon,now,accumulated_installed_g_co2_eq,accumulated_dependable_g_co2_eq,operating_hours,id,suggested_area,geometry
island_grid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Luzon,Off-Grid,AGUADA*,Oil-Based,Diesel,NaN,0.012,0.011,NaN,Masbate,"Cataingan, Masbate",11.986010,123.799202,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,Aug-2010,2010.0,850.0,1000.0,8760.0,89352000,81906000,NaN,2021,982872000,900966000,5,16f63e94-236e-43ce-8388-4f0e7ba37f17,True,POINT (123.79920 11.98601)
Luzon,Off-Grid,AMUTAG MGE1*,Oil-Based,Diesel,NaN,0.012,0.011,NaN,Masbate,"Aroroy, Masbate",12.468981,123.187455,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,Dec-2009,2009.0,850.0,1000.0,8760.0,89352000,81906000,NaN,2021,1072224000,982872000,5,2aadbc74-679d-4101-9058-38c8627b2185,True,POINT (123.18745 12.46898)
Luzon,Off-Grid,AMUTAG MGE2*,Oil-Based,Diesel,NaN,0.012,0.011,NaN,Masbate,"Aroroy, Masbate",12.468981,123.187455,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,Dec-2009,2009.0,850.0,1000.0,8760.0,89352000,81906000,NaN,2021,1072224000,982872000,5,f4d4b05e-ac03-43cf-9130-281b5bf76833,True,POINT (123.18745 12.46898)
Luzon,Off-Grid,BAANG(MOBO)*,Oil-Based,Diesel,NaN,0.006,0.005,NaN,Masbate,"Mobo, Masbate",12.264440,123.549202,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,May-2010,2010.0,850.0,1000.0,8760.0,44676000,37230000,NaN,2021,491436000,409530000,5,2a51f5c9-f992-4556-8ecf-ea39535907cc,True,POINT (123.54920 12.26444)
Luzon,Off-Grid,BAAO*,Oil-Based,Diesel,NaN,0.006,0.005,NaN,Masbate,"Baleno, Masbate",12.409210,123.381844,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,Aug-2009,2009.0,850.0,1000.0,8760.0,44676000,37230000,NaN,2021,536112000,446760000,5,e06a22cd-a7f3-486a-866b-889e49bc60a2,True,POINT (123.38184 12.40921)


In [10]:
for idx, projects in gdf.head().iterrows():
    point         = projects.geometry
    polygon_index = ph_cluster_km.distance(point).sort_values().index[0]
    gdf.loc[idx,"PVout"]      = ph_cluster_gmm.loc[polygon_index,"PVout"]    
    gdf.loc[idx,"Cluster"]    = ph_cluster_gmm.loc[polygon_index,"Cluster"]
    gdf.loc[idx,"lit_per"]    = ph_cluster_gmm.loc[polygon_index,"lit_per"]
    gdf.loc[idx,"Population"] = ph_cluster_gmm.loc[polygon_index,"Population"]
    gdf.loc[idx,"ADM3_EN"]    = ph_cluster_gmm.loc[polygon_index,"ADM3_EN"]    
    gdf.loc[idx,"ADM3_PCODE"] = ph_cluster_gmm.loc[polygon_index,"ADM3_PCODE"]
    gdf.loc[idx,"ADM3ALT1EN"] = ph_cluster_gmm.loc[polygon_index,"ADM3ALT1EN"]
    gdf.loc[idx,"ADM2_EN"]    = ph_cluster_gmm.loc[polygon_index,"ADM2_EN"]
    gdf.loc[idx,"ADM1_EN"]    = ph_cluster_gmm.loc[polygon_index,"ADM1_EN"]

In [11]:
gdf.head()

,connection_type,facility_name,category,subcategory,type,installed_capacity_mw,dependable_capacity_mw,number_of_units,province,municipality,latitude,longitude,region,operator,owner_ippa,fit_approved_for_re,owner_type,type_of_contract,date_commisioned_commercial,commisioned/commercial_year,c_kwh,mw_to_kw,no_of_hours,installed_g_co2_eq,dependable_g_co2_eq,icon,now,accumulated_installed_g_co2_eq,accumulated_dependable_g_co2_eq,operating_hours,id,suggested_area,geometry,PVout,Cluster,lit_per,Population,ADM3_EN,ADM3_PCODE,ADM3ALT1EN,ADM2_EN,ADM1_EN
island_grid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Luzon,Off-Grid,AGUADA*,Oil-Based,Diesel,NaN,0.012,0.011,NaN,Masbate,"Cataingan, Masbate",11.986010,123.799202,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,Aug-2010,2010.0,850.0,1000.0,8760.0,89352000,81906000,NaN,2021,982872000,900966000,5,16f63e94-236e-43ce-8388-4f0e7ba37f17,True,POINT (123.79920 11.98601),1365.278119,1.0,38.371993,104263.813961,Aroroy,PH054101000,None,Masbate,Region V
Luzon,Off-Grid,AMUTAG MGE1*,Oil-Based,Diesel,NaN,0.012,0.011,NaN,Masbate,"Aroroy, Masbate",12.468981,123.187455,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,Dec-2009,2009.0,850.0,1000.0,8760.0,89352000,81906000,NaN,2021,1072224000,982872000,5,2aadbc74-679d-4101-9058-38c8627b2185,True,POINT (123.18745 12.46898),1365.278119,1.0,38.371993,104263.813961,Aroroy,PH054101000,None,Masbate,Region V
Luzon,Off-Grid,AMUTAG MGE2*,Oil-Based,Diesel,NaN,0.012,0.011,NaN,Masbate,"Aroroy, Masbate",12.468981,123.187455,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,Dec-2009,2009.0,850.0,1000.0,8760.0,89352000,81906000,NaN,2021,1072224000,982872000,5,f4d4b05e-ac03-43cf-9130-281b5bf76833,True,POINT (123.18745 12.46898),1365.278119,1.0,38.371993,104263.813961,Aroroy,PH054101000,None,Masbate,Region V
Luzon,Off-Grid,BAANG(MOBO)*,Oil-Based,Diesel,NaN,0.006,0.005,NaN,Masbate,"Mobo, Masbate",12.264440,123.549202,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,May-2010,2010.0,850.0,1000.0,8760.0,44676000,37230000,NaN,2021,491436000,409530000,5,2a51f5c9-f992-4556-8ecf-ea39535907cc,True,POINT (123.54920 12.26444),1365.278119,1.0,38.371993,104263.813961,Aroroy,PH054101000,None,Masbate,Region V
Luzon,Off-Grid,BAAO*,Oil-Based,Diesel,NaN,0.006,0.005,NaN,Masbate,"Baleno, Masbate",12.409210,123.381844,5,National Power Corporation - Small Power Utili...,National Power Corporation - Small Power Utili...,NaN,NPC,NaN,Aug-2009,2009.0,850.0,1000.0,8760.0,44676000,37230000,NaN,2021,536112000,446760000,5,e06a22cd-a7f3-486a-866b-889e49bc60a2,True,POINT (123.38184 12.40921),1365.278119,1.0,38.371993,104263.813961,Aroroy,PH054101000,None,Masbate,Region V


In [12]:
doe_approved_solar_projects_energy_availability_clusters = gdf[gdf['category'] == 'Solar']
doe_approved_solar_projects_energy_availability_clusters.head()

,connection_type,facility_name,category,subcategory,type,installed_capacity_mw,dependable_capacity_mw,number_of_units,province,municipality,latitude,longitude,region,operator,owner_ippa,fit_approved_for_re,owner_type,type_of_contract,date_commisioned_commercial,commisioned/commercial_year,c_kwh,mw_to_kw,no_of_hours,installed_g_co2_eq,dependable_g_co2_eq,icon,now,accumulated_installed_g_co2_eq,accumulated_dependable_g_co2_eq,operating_hours,id,suggested_area,geometry,PVout,Cluster,lit_per,Population,ADM3_EN,ADM3_PCODE,ADM3ALT1EN,ADM2_EN,ADM1_EN
island_grid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Luzon,Grid-Connected,CALATAGAN SOLAR,Solar,NaN,Ground Mounted Solar PVs,63.3,50.6,NaN,Batangas,"Calatagan and Balayan, Batangas",13.89066,120.687019,4-A,Solar Philippines Calatagan Corporation,Solar Philippines Calatagan Corporation,YES,NON-NPC/IPP,NaN,Mar-2016,2016.0,20.0,1000.0,8760.0,11090160000,8865120000,NaN,2021,55450800000,44325600000,24,bdac6570-8d7d-49a9-be6c-d953cf0e12f3,NaN,POINT (120.68702 13.89066),1365.278119,1.0,38.371993,104263.813961,Aroroy,PH054101000,None,Masbate,Region V
Luzon,Grid-Connected,CLARK SOLAR,Solar,NaN,Ground Mounted Solar PVs,22.3,17.8,NaN,Pampanga,"Prince Balagtas Extension, Clark Special Econo...",15.24685,120.580261,3,Enfinity Philippines Renewable Resources Inc.,Enfinity Philippines Renewable Resources Inc.,YES,NON-NPC/IPP,NaN,Mar-2016,2016.0,20.0,1000.0,8760.0,3906960000,3118560000,NaN,2021,19534800000,15592800000,24,0787d3a2-da67-403c-8c9d-0ad7cac1b72c,NaN,POINT (120.58026 15.24685),1365.278119,1.0,38.371993,104263.813961,Aroroy,PH054101000,None,Masbate,Region V
Luzon,Grid-Connected,CONCEPCION 1 SOLAR,Solar,NaN,Ground Mounted Solar PVs,20.7,16.6,NaN,Tarlac,"Concepcion , Tarlac",15.33112,120.587868,3,Solar Philippines Tarlac Corporation,Solar Philippines Tarlac Corporation,NO,NON-NPC/IPP,NaN,Sep-2019,2019.0,20.0,1000.0,8760.0,3626640000,2908320000,NaN,2021,7253280000,5816640000,24,00603154-c501-4d09-96ac-284e267965dc,NaN,POINT (120.58787 15.33112),1365.278119,1.0,38.371993,104263.813961,Aroroy,PH054101000,None,Masbate,Region V
Luzon,Grid-Connected,CONCEPCION 2 SOLAR,Solar,NaN,Ground Mounted Solar PVs,70.9,56.7,NaN,Tarlac,"Concepcion , Tarlac",15.33112,120.587868,3,Solar Philippines Tarlac Corporation,Solar Philippines Tarlac Corporation,NO,NON-NPC/IPP,NaN,Feb-2020,2020.0,20.0,1000.0,8760.0,12421680000,9933840000,NaN,2021,12421680000,9933840000,24,b76bde70-5219-4d88-ab11-74fe7cd7c47b,NaN,POINT (120.58787 15.33112),1365.278119,1.0,38.371993,104263.813961,Aroroy,PH054101000,None,Masbate,Region V
Luzon,Grid-Connected,CURRIMAO SOLAR,Solar,NaN,Ground Mounted Solar PVs,20.0,16.0,NaN,Ilocos Norte,"Currimao, Ilocos Norte",18.01882,120.465774,1,Mirae Asia Energy Corporation,Mirae Asia Energy Corporation,YES,NON-NPC/IPP,NaN,Feb-2016,2016.0,20.0,1000.0,8760.0,3504000000,2803200000,NaN,2021,17520000000,14016000000,24,108f2790-d682-47e5-b2cf-0c08ecb1ea26,NaN,POINT (120.46577 18.01882),1365.278119,1.0,38.371993,104263.813961,Aroroy,PH054101000,None,Masbate,Region V


In [13]:
cluster_summary = doe_approved_solar_projects_energy_availability_clusters.groupby('Cluster').agg({'PVout': ['mean', 'min', 'max','count']})
cluster_summary

PVout                                
                mean          min          max count
Cluster                                             
1.0      1365.278119  1365.278119  1365.278119    29

In [14]:
doe_approved_solar_projects_energy_availability_clusters.shape

(48, 42)

In [15]:
gdf.to_csv("data/results/doe_approved_projects_energy_integrated_clusters.csv")